In [ ]:
import torch
import soundfile as sf
import pandas as pd
import numpy as np
import resampy
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from transformers import BitsAndBytesConfig
from pydub import AudioSegment

device = "cuda" if torch.cuda.is_available() else "cpu"

whisper_path = "openai/whisper-small"
processor_stt = WhisperProcessor.from_pretrained(whisper_path)
model_stt = WhisperForConditionalGeneration.from_pretrained(whisper_path).to(device)

base_model = 'beomi/Llama-3-Open-ko-8B-Instruct-preview'
adapter_path = './model/model1_0724_ver1/checkpoint-2700'

torch_dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.get_device_capability()[0]>=8) else torch.float16
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=False
)

base = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=quant_config,
                                            attn_implementation="eager", trust_remote_code=True)
model = PeftModel.from_pretrained(base, adapter_path)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


audio_path = "../file/sample_2.wav"
audio, sr = sf.read(audio_path)

if len(audio.shape) > 1:
    audio = np.mean(audio, axis=1)


if sr != 16000:
    audio = resampy.resample(audio, sr, 16000)
    sr = 16000
audio = audio.astype("float32")

chunk_sec = 5
chunk_samples = chunk_sec * sr
num_chunks = max(1, len(audio) // chunk_samples)
chunks = np.array_split(audio, num_chunks)

results = []

for idx, chunk in enumerate(chunks):

    inputs = processor_stt(chunk, sampling_rate=sr, return_tensors="pt").input_features.to(device)
    generated_ids = model_stt.generate(input_features=inputs)
    dialect_text = processor_stt.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    prompt = f"다음 문장을 제주 방언으로 번역해줘.\n{dialect_text}\n=> "
    inputs_llm = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(**inputs_llm, max_new_tokens=80)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    results.append({
        "STT": dialect_text,
        "Translation": translated_text
    })

df = pd.DataFrame(results)
print(df)


Loading checkpoint shards: 100%|██████████| 4/4 [00:16<00:00,  4.08s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

                                        STT  \
0              지난주말 폭소리 내리는 가운데에도 음박담요를 두르고   
1                한강동 관조합을 지키며 윤대통령 체포를 촉구했던   
2    이른바 키세스 시위되는 오늘 공수차 앞에서 기자회견을 열고 있습니다.   
3                공소처에 적극적인 체포영장 집행을 요구했습니다.   
4                  키스에서 시위되는 지난 3일 공수처가 체포형   
5                              5시간 만에 철수한 뒤   
6             2박 3일간 한남동 관저 하플 지키며 체포를 촉구했던   
7                       시민들입니다. 키스스 대학생 시위대   
8                이들은 지난 3일 윤석열의 신병 확보가 가능했고   
9                       공주처가 5시간 만에 체포를 포기한   
10                   참으로써 체포 의지가 없었음을 보여줬다며   
11                           공권력이 내란 숙에 바라가   
12                    무기력한 모습을 보여줬다고 말했습니다.   

                                          Translation  
0   다음 문장을 제주 방언으로 번역해줘.\n 지난주말 폭소리 내리는 가운데에도 음박담요...  
1   다음 문장을 제주 방언으로 번역해줘.\n 한강동 관조합을 지키며 윤대통령 체포를 촉...  
2   다음 문장을 제주 방언으로 번역해줘.\n 이른바 키세스 시위되는 오늘 공수차 앞에서...  
3   다음 문장을 제주 방언으로 번역해줘.\n 공소처에 적극적인 체포영장 집행을 요구했습...  
4   다음 문장을 제주 방언으로 번역해줘.\n 키스에서 시위되는 지난 3일 공수처가 체포...  
5   다

In [11]:
df['제주방언']=df['Translation'].str.split('=>').str[1].tolist()
df.drop(columns='Translation',inplace=True)
df['제주방언']=df['제주방언'].str.split('<').str[0].tolist()

In [12]:
df

,STT,제주방언
0,지난주말 폭소리 내리는 가운데에도 음박담요를 두르고,지난주말 폭소리 내리는 가운데에도 음박담요를 두르고이
1,한강동 관조합을 지키며 윤대통령 체포를 촉구했던,한강동 관조합을 지키멍 윤대통령 체포를 촉구했던
2,이른바 키세스 시위되는 오늘 공수차 앞에서 기자회견을 열고 있습니다.,이른바 키세스 시위되는 오늘 공수차 앞에서 기자회견을 열고 있수다게
3,공소처에 적극적인 체포영장 집행을 요구했습니다.,공소처에 적극적인 체포영장 집행을 요구했수다게
4,키스에서 시위되는 지난 3일 공수처가 체포형,키스에서 시위되는 지난 3일 공수처가 체포형이
5,5시간 만에 철수한 뒤,5시간 만에 철수한 뒤이
6,2박 3일간 한남동 관저 하플 지키며 체포를 촉구했던,2박 3일간 한남동 관저 하플 지키멍 체포를 촉구했던
7,시민들입니다. 키스스 대학생 시위대,시민들입니다. 키스스 대학생 시위대이
8,이들은 지난 3일 윤석열의 신병 확보가 가능했고,이들은 지난 3일 윤석열의 신병 확보가 가능했주게
9,공주처가 5시간 만에 체포를 포기한,공주처가 5시간 만에 체포를 포기핸


In [4]:
df.to_excel('stt_sample_2.xlsx')